### This notebook is to test the mulidimensional priors and how to serialise them. I want to have a very convientient method which is more or less based on the one that is currently disscussed in the PR. 
### Everything here is developed in the gammapy branch IRF_model 

In [1]:
#get_ipython().system('jupyter nbconvert --to script 1-Nui_Par_Fitting.ipynb')
import pyximport

pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from gammapy.maps import Map
from astropy.coordinates import SkyCoord, Angle
from gammapy.modeling import Fit,  Parameters, Covariance , Parameter
from gammapy.datasets import MapDataset ,Datasets, FluxPointsDataset
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
    PointSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SpatialModel,
    FoVBackgroundModel,
    PiecewiseNormSpectralModel,
)
from gammapy.estimators import TSMapEstimator, ExcessMapEstimator
from gammapy.estimators import FluxPoints, FluxPointsEstimator

from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys
sys.path.append('../../')
import Dataset_load 

from  Dataset_Setup import Setup, GaussianCovariance_matrix

In [2]:
from matplotlib import rc
import matplotlib.pylab as plt

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

config = Dataset_load.load_config()
colors = config["colors"]["three"]
import ast

colors[1] = ast.literal_eval(colors[1])
colors[2] = ast.literal_eval(colors[2])
colors[3] = ast.literal_eval(colors[3])

#import seaborn as sns


alpha_nui = 0.99
alpha_st = 0.99
legendsscatter = ["Asimov w/o nui.", "Asimov with nui."]
legends = ["", ""]

alpha_rnd = 0.2
alpha_rnd_nui = 0.2

nbins = 20

In [3]:
scaled_amplitude = Parameter("amplitude", value=1e-12)
#lambda_ = Parameter("lambda_", value=1/60)

dataset_input = Dataset_load.create_asimov(
    model="pl", 
    source="PKSflare", 
    parameters=Parameters([scaled_amplitude])
)
source_model = dataset_input.models[0].copy()
emask = dataset_input.mask.data.sum(axis=2).sum(axis=1)>0
energy = dataset_input.geoms['geom'].axes[0].center
energy_mask = dataset_input.geoms['geom'].axes[0].center[emask]

loaded dataset:
/home/katrin/Documents/nuisance_summary/PKS_flare/HESS_public/dataset-simulated-pl.fits.gz


## Systematic Settings

In [4]:
# bkg sys
magnitude = 10 # in percent
corrlength = 2 # correlation length  in terms of energy bins
seed = 38

bias = 0.1
resolution= 0.
norm = 0.1
tilt = 0.02

In [5]:
setup = Setup(dataset_input=dataset_input)
#setup.set_up_irf_sys(bias, resolution, norm, tilt)
dataset_asimov, dataset_asimov_N = setup.run()
# irf model
setup.set_irf_model(dataset_asimov_N)
dataset_asimov_N.models.parameters['resolution'].frozen = True
dataset_asimov_N.irf_model.parameters['tilt'].frozen = False


setup.set_irf_prior(dataset_asimov_N, bias, resolution, norm, tilt)
# piece wise model
# remove old bkg model
setup.unset_model(dataset_asimov_N, FoVBackgroundModel)
setup.set_piecewise_bkg_model(dataset_asimov_N)
setup.set_bkg_prior(dataset_asimov_N, magnitude, corrlength)
# energy of the following parameters smaller than ethrshold
dataset_asimov_N.background_model.parameters['norm0'].frozen = True
dataset_asimov_N.background_model.parameters['norm1'].frozen = True
dataset_asimov_N.background_model.parameters['norm2'].frozen = True
dataset_asimov_N.background_model.parameters['norm3'].frozen = True
print(dataset_asimov_N.models)

DatasetModels

Component 0: SkyModel

  Name                      : aEaZD8Bs
  Datasets names            : None
  Spectral model type       : PowerLawSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    index                         :      2.300   +/-    0.00             
    amplitude                     :   1.00e-12   +/- 0.0e+00 1 / (TeV s cm2)
    reference             (frozen):      1.000       TeV         
    lon_0                         :    329.680   +/-    0.00 deg         
    lat_0                         :    -30.222   +/-    0.00 deg         

Component 1: IRFModels

  Name                      : XdaPUnNU-irf
  Datasets names            : XdaPUnNU
  EReco  model type         : ERecoIRFModel
  Eff area  model type      : EffAreaIRFModel
  PSF model type            : 
  Parameters:
    bias                          :      0.000   +/-    0.00             
    resolution            (frozen):      0.000                

In [6]:
%%time
fit_sys = Fit()
#fit_sys.run(dataset_asimov_N)
dataset_asimov_N.models.write("test.yaml", overwrite = True)


CPU times: user 128 ms, sys: 923 µs, total: 129 ms
Wall time: 126 ms


## Read in the whole model: This is the endgoal :) 

In [7]:
model_read = Models.read("test.yaml")

In [8]:
print(model_read.parameters['norm1'].prior._modelparameters)
print(model_read.parameters['norm1'].prior._covariance_matrix[0])

[  1406.14217965  -5592.09282613  12491.02753087 -20782.32194979
  28818.48769322 -35336.17906024  39694.7381362  -41793.40528998
  41874.06116254 -40336.0491627   37608.62290578 -34081.34436728
  30076.68083831 -25848.00330881  21590.33128959 -17455.70570167
  13568.33214024 -10036.4015403    6958.10620966  -4419.45822164
   2482.09439609  -1161.84654458    405.0898453     -79.3291133 ]
